In [1]:
import json
from pathlib import Path
from docile.dataset import Dataset
from docile.dataset import Field
from docile.tools.dataset_browser import DatasetBrowser

In [3]:
DATASET_PATH = Path("/app/data/docile/")
# dataset = Dataset("trainval", DATASET_PATH, load_annotations=True, load_ocr=True)

In [7]:
docile_trainval = Dataset("trainval", DATASET_PATH)

Loading documents for docile:trainval: 100%|██████████| 5680/5680 [00:41<00:00, 137.19it/s]


In [10]:
from collections import Counter

f_newlines = Counter()
f_all = Counter()
for doc in docile_trainval:
    for f in doc.annotation.fields + doc.annotation.li_fields:
        f_all[f.fieldtype] += 1
        if "\n" in f.text:
            f_newlines[f.fieldtype] += 1

In [18]:
multiline_fields = []
for ft, cn in f_newlines.most_common(None):
    ca = f_all[ft]
    percent = cn/ca*100
    if percent > 0.5:
        print(f"{ft}: {cn}/{ca} ({percent})")
        multiline_fields.append(ft)

vendor_address: 5612/6634 (84.59451311425987)
customer_billing_address: 5174/5249 (98.5711564107449)
line_item_description: 2151/28617 (7.516511164692316)
customer_billing_name: 1468/6142 (23.90100944317812)
vendor_name: 823/7354 (11.191188468860485)
customer_delivery_address: 340/350 (97.14285714285714)
customer_other_address: 328/344 (95.34883720930233)
line_item_code: 278/8904 (3.1221922731356693)
customer_delivery_name: 127/378 (33.5978835978836)
customer_other_name: 84/1463 (5.741626794258373)
payment_terms: 17/2295 (0.7407407407407408)
line_item_person_name: 7/496 (1.411290322580645)


In [19]:
from docile.dataset import KILE_FIELDTYPES, LIR_FIELDTYPES

for f in KILE_FIELDTYPES + LIR_FIELDTYPES:
    if f.endswith("address") or f.endswith("name") or f.endswith("description"):
        if f not in multiline_fields:
            print(f"Error, fieldtype {f} missing in multiline fields!")

In [4]:
docile_test = Dataset("test", DATASET_PATH)

Loading documents for docile:test: 100%|██████████| 1000/1000 [00:05<00:00, 190.30it/s]


In [ ]:
cnt_pages_all = 0
cnt_pages_wrong = 0
for doc in dataset:
    for page in range(doc.page_count):
        ok = any(f.page == page for f in doc.annotation.li_fields)
        header_ok = any(f.page == page for f in doc.annotation.li_headers)
        cnt_pages_all += 1
        if ok and not header_ok:
            cnt_pages_wrong += 1
            # print(doc)
            # print(page)
            # print(ok, header_ok)

In [ ]:
print(cnt_pages_wrong)
print(cnt_pages_all)

# Display dataset annotations

In [ ]:
DatasetBrowser([dataset["00aa98164d264f4e924f55a9"]])

# Display with predictions

In [ ]:
from docile.dataset import load_predictions

# KILE_PREDICTION_PATH=Path("/app/data/baselines/predictions_test_only_docile/layoutlmv3_base_315750/test_predictions_KILE.json")
# LIR_PREDICTION_PATH=Path("/app/data/baselines/predictions_test_only_docile/layoutlmv3_base_315750/test_predictions_LIR.json")
KILE_PREDICTION_PATH=Path("/app/data/test_rir_format_conversion/test_predictions_KILE_perfect.json")
LIR_PREDICTION_PATH=Path("/app/data/test_rir_format_conversion/test_predictions_LIR_perfect.json")

kile_predictions = load_predictions(KILE_PREDICTION_PATH)
lir_predictions = load_predictions(LIR_PREDICTION_PATH)

In [ ]:
DatasetBrowser(dataset, kile_predictions=kile_predictions, lir_predictions=lir_predictions);
# you can also display just single task predictions, e.g.:
# DatasetBrowser(dataset, lir_predictions=lir_predictions);

# Display with matching from evaluation

In [20]:
from docile.evaluation.evaluate import EvaluationResult

# KILE_EVALUATION_PATH=Path("example_val_results_KILE.json")
# LIR_EVALUATION_PATH=Path("example_val_results_LIR.json")

# EVALUATION_PATH=Path("/app/data/rir-predictions/stack_predict_v4_no_collectors/test_results.json")
EVALUATION_PATH=Path("/app/data/rir-predictions/dag-230629T114502-1c449bbf-loving-carson/test_results.json")


evaluation_result = EvaluationResult.from_file(EVALUATION_PATH)
kile_matching = evaluation_result.task_to_docid_to_matching["kile"]
lir_matching = evaluation_result.task_to_docid_to_matching["lir"]

In [26]:
from docile.dataset import Document
DatasetBrowser([Document("synthetic-4d85703835b14d0c9cf006be-000", DATASET_PATH)])#, kile_matching=kile_matching, lir_matching=lir_matching)